# Supervised Fine-Tuning (SFT) with Serverless customization on SageMaker AI

## Lab 2 - Fine-Tune LLM

In this notebook, we are going to run a Serverless Supervised Fine-Tuning job on SageMaker AI

***

### Prerequistes

#### Setup and dependencies

In [2]:
import boto3
from sagemaker.core.helper.session_helper import Session, get_execution_role

sess = Session()
sagemaker_session_bucket = None

if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

s3_client = boto3.client("s3")
sess = Session(default_bucket=sagemaker_session_bucket)
sm_client = boto3.client("sagemaker", region_name=sess.boto_region_name)
bucket_name = sess.default_bucket()
default_prefix = sess.default_bucket_prefix

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

[01/22/26 11:13:05] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=421703;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=323223;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/botocore/credentials.py#1392\1392]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/bpistone/Library/Application Support/sagemaker/config.yaml


[01/22/26 11:13:08] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=401115;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=817392;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/botocore/credentials.py#1392\1392]8;;\

[01/22/26 11:13:09] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=398086;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=560064;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/botocore/credentials.py#1392\1392]8;;\

sagemaker role arn: arn:aws:iam::691148928602:role/mlops-sagemaker-execution-role
sagemaker bucket: sagemaker-us-east-1-691148928602
sagemaker session region: us-east-1


In [3]:
from sagemaker.ai_registry.dataset import DataSet

base_model_id = "huggingface-llm-qwen2-5-7b-instruct"

training_dataset = DataSet.get(name="medical-o1-reasoning-sft-train")
val_dataset = DataSet.get(name="medical-o1-reasoning-sft-val")

if default_prefix:
    output_path = f"s3://{bucket_name}/{default_prefix}/{base_model_id}"
else:
    output_path = f"s3://{bucket_name}/{base_model_id}"

os.environ["SAGEMAKER_MLFLOW_CUSTOM_ENDPOINT"] = (
    f"https://mlflow.sagemaker.{sess.boto_region_name}.app.aws"
)

[01/22/26 11:13:10] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=934555;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=542081;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/botocore/credentials.py#1392\1392]8;;\

[01/22/26 11:13:11] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=283177;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=290254;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py#92\92]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     SageMaker session not provided. Using default Session.                  ]8;id=119438;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=893901;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

[01/22/26 11:13:12] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=141006;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=136609;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py#92\92]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     SageMaker session not provided. Using default Session.                  ]8;id=518951;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=217063;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

***

### Create Model Package Group

In [4]:
from botocore.exceptions import ClientError

model_package_group_name = f"{base_model_id}-mpg"
model_package_group_description = ""

operation_input_args = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": model_package_group_description,
}

try:
    sm_client.create_model_package_group(**operation_input_args)
except ClientError as e:
    if e.response["Error"]["Code"] == "ValidationException":
        print("Model Package Group already exists")
    else:
        raise

### Run Severless Job

In [5]:
from sagemaker.train.common import TrainingType
from sagemaker.train.sft_trainer import SFTTrainer

In [6]:
trainer = SFTTrainer(
    model=base_model_id,
    training_type=TrainingType.LORA,
    model_package_group=model_package_group_name,
    training_dataset=training_dataset,
    validation_dataset=val_dataset,
    s3_output_path=output_path,
    sagemaker_session=sess,
    role=role,
    accept_eula=True,
)

[01/22/26 11:13:36] INFO     Runs on sagemaker prod, region:us-east-1                                  ]8;id=959353;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=935042;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#354\354]8;;\

Print Hyperparameters

In [8]:
from rich import print as rprint
from rich.pretty import pprint

print("Default Finetuning options:")
pprint(trainer.hyperparameters.to_dict())

Default Finetuning options:


{
│   'global_batch_size': '64',
│   'learning_rate': '0.0001',
│   'lr_warmup_ratio': '0.1',
│   'max_epochs': '3',
│   'mlflow_run_id': '',
│   'mlflow_tracking_uri': '',
│   'model_name_or_path': 'Qwen/Qwen2.5-7B-Instruct',
│   'name': 'example-name-lad5y',
│   'results_directory': '',
│   'resume_from_path': '',
│   'train_val_split_ratio': '0.9'
}

Override Hyperparameters

In [9]:
trainer.hyperparameters.learning_rate = 0.0001
trainer.hyperparameters.global_batch_size = 64
trainer.hyperparameters.max_epochs = 3
trainer.hyperparameters.lr_warmup_ratio = 0.1

In [10]:
print("\nModified/user defined options:")
pprint(trainer.hyperparameters.to_dict())


Modified/user defined options:


{
│   'global_batch_size': '64',
│   'learning_rate': '0.0001',
│   'lr_warmup_ratio': '0.1',
│   'max_epochs': '3',
│   'mlflow_run_id': '',
│   'mlflow_tracking_uri': '',
│   'model_name_or_path': 'Qwen/Qwen2.5-7B-Instruct',
│   'name': 'example-name-lad5y',
│   'results_directory': '',
│   'resume_from_path': '',
│   'train_val_split_ratio': '0.9'
}

In [11]:
from rich import print as rprint
from rich.pretty import pprint

training_job = trainer.train(wait=False)

TRAINING_JOB_NAME = training_job.training_job_name

pprint(training_job)

[01/22/26 11:13:46] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=833974;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=242142;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py#92\92]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Training Job Name:                                                  ]8;id=374588;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/sft_trainer.py\sft_trainer.py]8;;\:]8;id=877708;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/sft_trainer.py#203\203]8;;\
                             huggingface-llm-qwen2-5-7b-instruct-sft-20260122111346                                

[01/22/26 11:13:48] INFO     Using first available ready MLflow app:                          ]8;id=189439;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=991408;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#136\136]8;;\
                             arn:aws:sagemaker:us-east-1:691148928602:mlflow-app/app-EJODF3XH                      
                             6IEF                                                                                  

                    INFO     MLflow resource ARN:                                             ]8;id=390713;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=65323;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#586\586]8;;\
                             arn:aws:sagemaker:us-east-1:691148928602:mlflow-app/app-EJODF3XH                      
                             6IEF                                                                                  

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/bpistone/Library/Application Support/sagemaker/config.yaml


                    INFO     Creating training_job resource.                                     ]8;id=800690;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=15709;file:///Users/bpistone/miniconda3/envs/python312-sm3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

TrainingJob(
│   training_job_name='huggingface-llm-qwen2-5-7b-instruct-sft-20260122111346',
│   training_job_arn='arn:aws:sagemaker:us-east-1:691148928602:training-job/huggingface-llm-qwen2-5-7b-instruct-sft-20260122111346',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   model_artifacts=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   training_job_output=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   training_job_status='InProgress',
│   secondary_status='Starting',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   hyper_parameters={
│   │   'global_batch_size': '64',
│   │   'learning_rate': '0.0001',
│   │   'lr_warmup_ratio': '0.1',
│   │   'max_epochs': '3',
│   │   'model_name_or_path': 'Qwen/Qwen2.5-7B-Instruct',
│   │   'name': 'example-name-lad5y',
│   │   'train_val_split_ratio': '0.9'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   role_arn='arn:aws:iam::691148928602:role/mlops-sagemaker-execution-role',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-east-1:691148928602:hub-content/UV1IAI517N7DDV758JI1AKSBLONNQ4SEMJIOLN7HIAPQ1PDDVOQG/DataSet/medical-o1-reasoning-sft-train/1.0.0'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   enable_ffm=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>
│   │   ),
│   │   Channel(
│   │   │   channel_name='validation',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-east-1:691148928602:hub-content/UV1IAI517N7DDV758JI1AKSBLONNQ4SEMJIOLN7HIAPQ1PDDVOQG/DataSet/medical-o1-reasoning-sft-val/1.0.0'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   │   enable_ffm=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://sagemaker-us-east-1-691148928602/huggingface-llm-qwen2-5-7b-instruct',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   disable_model_upload=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>
│   ),
│   resource_config=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x1294c8f80>,